# Working with Dataloop.ai - a platform workshop


## Introduction

This document will provide instructions on how to get started with dataloop platform and tthe hands on experience on most on the platform components.


You will cover a full active learning pipeline by the end of the workshop .



## Step 1. Set up your Data environment:
In this step we will create a dataset that will be used afterwards to manage the data and automate the pipeline,along with creating a recipy that will define the annotation prossess and help in searcing / filtering  the annotated data.



### Step 1.1 Create a Dataset :
From the **Data page**, click on **'create dataset'**, give it a name and select the **'file system'** storage option.

### Step 1.2 Edit the Ontology :
From the Ontology page
 double click and open the previously created dataset ontology .

1.   Double click and open the previously created dataset ontology.
2.   In labels and attributes, click the import labels icon, and import the **labels.txt** file from the attached files.
3. In instructions click on the **Annotation Instructions** and upload the instructions.pdf in files for the annotation team.
4.In **Annotations Verification**, upload the ? to upload or not?


>NOTE: You can create multiple ontologies and switch between them for the dataset based on the mission.



## Step 2. Build basic pipeline flow:
In this step we will create basic pipeline flow, that contains a dataset, small script and an annotation task.


### Step 2.1 Create a pipeline :
From the **Pipelines page**, click on **create pipeline**, then **Start from scratch** ,give it a name and create.


### Step 2.2 Edit the pipeline :
From the elements panel :

1.   Drag and drop the **Data element - Dataset**.
2.   Click on the added element and edit it's info panel on the right side, by selecting the previusly created dataset,and turning the trigger toggle on for **once an item is created**.
3. Drag and drop the **Automation element - Code** and connect the previous element with it.
4.Click on the added element and edit it's info panel :
  * Node Name as - **Random Box**
  * Application Name as - **YourName-add-random**
  * Source Code Type as - **Code Editor**
  * Open Code Editor and copy paste the following code and save it.
5. Drag and drop the **Automation element - FAAS ** and connect the previous element with it.
6.On the info panel,search for the **Flip_item** pre-added function,that will flip the item before passing it to the labeling task.
6.Drag and drop the **Workflow element - Labeling** and connect the previous element with it..
7.From the info panel,click on **Create Task** and fill out the task requirements then create the task .

>NOTE:The Dataset selected in the dataset node must match the dataset configured in the labeling task.

In [ ]:
import dtlpy as dl

class ServiceRunner:

    def run(self, item):
        builder = item.annotations.builder()
        builder.add(annotation_definition=dl.Box(top=10,left=10,bottom=100, right=100,label='person'))
        item.annotations.upload(builder)
        return item


### Step 2.3 Run and execute the pipeline :
1. Click on **Start Pipeline** to start the pipeline.
2. Open the Dataset that you have created previously from the **Data Page** and upload data from the **Sample_Data** attached file.
3. Track the items cycles and untill they get to the labeling task .
4. Go the the Labeling page and open your rtask to complete the cycle.

>NOTE:Uploading the items will trigger the pipeline to execute and create a cycle for each one of them, once the item is done in an element, it will be automatically moved to the next one.

## Step 3. Build an Active learning pipeline:
 The Dataloop's Active Learning Pipeline (ALP) is a powerful and customizable feature designed to automate and streamline the iterative model training process on unstructured data.
 The ability to a
 utomate the iterative process of fine-tuning your model, covering labeling, model creation, training, evaluation, and deployment of the best version.


### Step 3.1 Create a model :

1.   From the **Models page**, go to the **'AI Library'** and view the Public plications to see the several pre-installed and publicly available model architectures, which can be installed in your project and trained with your data to achieve pre-annotations of your labels.
2.   Click on the create model icon next to the **Yolov8 model** .
3.   Edit the Model name to **YourName-cloned-yolov8**.
4.   In **Data & Instructions** turn off the finetune option, since we are going to edit the training fields in the pipeline.
5.   Click on **Create Model**.

>NOTE:Dataloop icon next to the base models list, represents a public trained model from our AI library that you can easly clone and start from it.

## Step 3.2 Create the Active learning pipeline from a template:

1.   From the **Pipelines Page** click on **Create Pipeline** and then select the option to **Use a Template**.
2.   Search for the **Active Learning Template**,click on it and click on install to install it's application.
3.   Click on **Create pipeline** then go to the **Actions menu** to rename it
 as **YourName-ALP**.




## Step 3.3 Edit the Active learning pipeline:
The flow of Dataloop's active learning pipeline is divided into two flows:

1. Ground Truth Enrichment: Collecting train Data into the Ground Truth - The upper flow
2. Training a new version of your model based on the collected data - The lower flow

> Note : For more information about the pipeline nodes, see the [Pipeline Nodes Document](https://dataloop.ai/docs/pipeline-nodes).



### Step 3.3.1  Upper Flow: Collection of Ground-Truth Data:


1. Pipeline Trigger:  
By default, an **item_created** event trigger is set on the first node ('Dataset Node') to trigger the ALP on every item creation (following data upload/sync) in the dataset you set on the Dataset Node. This trigger can be modified so you can select different events or add a DQL filter to the trigger.

2. Dataset Node:   
Use a Dataset node as the starting point of your pipeline to ensure it only triggers events in that specific - **Select the dataset you've created in step 1**.
> Note :The Dataset selected in this node must match the dataset configured in the Annotation workflow task nodes after the predict node.

3. Predict Node:  
The active learning pipeline begins with the **Predict node**, where a pre-trained model generates annotations on unlabeled data, using the model set under **Best model variable**.

 *   Go to the **Pipeline Variables** and click on **Manage Variables**.
 *   Edit the **Best Model Variable** and add your **Model Id** as the value.
 *   Click the **v icon** to add it and **Save Changes**.


4. Human in The Loop (HITL):   
Once pre-annotation using your model is done, **data items move directly into human annotation workflow tasks according to your needs**.  
 The default ALP consists of a labeling task and a QA (Review) task to correct the model's pre-annotations. Customize the workflow section according to your needs and fully set up the task.

5. Split Data into Ground Truth Subsets - ML Data Split Node :   
It randomly splits the annotated data into three subset groups based on the given distribution percentage.   
By default, the distribution is: **80-10-10** for Train-Validation-Test subsets.  
Metadata tags will be added to the items' metadata under metadata.system.tags
 > For example:   
 {
  "metadata": {
    "system": {
      "tags": {
      "train": true
      }
    }
  }
}  
These metadata tags to be later easily found and used as DQL filter for the train and evaluation.

6. Dataset Node - Ground Truth:  
 Select the Ground Truth dataset to store the new data in.
 The data will be cloned from the dataset selected in the first pipeline node.
 > Note: If you want to use the same dataset as you set in the pipeline starting point (the first node), then you can either remove the current Dataset node or keep it there; it won't affect the data.

### Step 3.3.2  Lower Flow: Create, Train, Deploy New Model Version:


1. Have_new_items FAAS -   
 *   Drag and drop the **Automation element - FAAS** and connect the create new model element.
 *   From the info panel,search and select the **Have_new_items** pre-added function,that will wait every x items and execute the training flow.
 *   Define the item_count_threshold variable from the info panel .

2. Create a New Model Node:   
This node clones a base model using the variables set on the node inputs (dataset, training and validation subsets, and model configurations). This node outputs a model that is prepared for training.  
 The name of the new model entity is taken from the **New Model Name text box** in the node's panel.  
 > Note: If a model already exists with the same name, an incremental number will be automatically added as a suffix.

3. Train Model Node:   
The Train Model node executes the training process for the newly created model version.  
The model will be trained over the Ground-Truth dataset using the train and validation subsets that were saved on the model during its creation.

4. Evaluate Node - Base Model:   
The Evaluate Model node creates evaluation for the base model to make sure its evaluation is up-to-date, and based on the entire test subset of the Ground-Truth dataset (as defined in the variables).   
The evaluation will be used later for comparison with the newly created model ('Best model' variable).

5. Compare Models Node :  
The Compare Models node compares two models: a previously trained model and a newly trained model.


6. Update Variable Node :   
If the winning model is the new model (based on the comparison config), the Update Variable node will automatically deploy the model and update the **Best model** variable.  
Your pipeline will start using the new model immediately across the pipeline - every node that uses this variable (predict, create model).

## Step 4. Executing the Pipeline
Once you configure your Active Learning Pipeline:  

1.   Click **Start Pipeline** to activate it.  
2.   Trigger the pipeline :
 *   Automatically : By uploading items from the **Sample_Data** file to the triggered dataset.
 *   Manually trigger the pipeline over existing data by going to the Browser of the triggered dataset,filter the required data and click Dataset Actions > Run with pipeline > Select your pipeline.
3. Track the items in the pipeline and start working !



## Useful Links:
[Active Learning Pipeline Document](https://dataloop.ai/docs/active-learning-pipeline).
[Model Training Document](https://dataloop.ai/docs/model-training).
[Pipeline Nodes Document](https://dataloop.ai/docs/pipeline-nodes).


